In [1]:
import pandas as pd

In [81]:
df = pd.read_csv('test.csv', index_col="player_id")
df = df.melt(ignore_index=False).reset_index()
# df = df[df["value"] != 0]
print(df["value"].max())
df

94.0


,player_id,variable,value
0,86540,Action,7.0
1,144736,Action,0.0
2,547685,Action,0.0
3,554278,Action,0.0
4,561758,Action,17.0
...,...,...,...
52075,99704390,Strategy,0.0
52076,99723205,Strategy,1.0
52077,99766416,Strategy,0.0
52078,99812428,Strategy,2.0


In [75]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [82]:
from surprise import accuracy, Dataset, SVD, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 94))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["player_id", "variable", "value"]], reader)

trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

RMSE: 2.4213
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.5019  2.4348  2.5962  2.3366  2.5952  2.4930  0.0990  
MAE (testset)     0.9376  0.9333  0.9432  0.9155  0.9758  0.9411  0.0197  
Fit time          0.73    0.86    0.75    0.64    0.65    0.73    0.08    
Test time         0.16    0.10    0.12    0.09    0.08    0.11    0.03    


{'test_rmse': array([2.50185966, 2.43482745, 2.59622202, 2.33662734, 2.59522272]),
 'test_mae': array([0.93760415, 0.93328845, 0.94322024, 0.91549237, 0.97578726]),
 'fit_time': (0.7281553745269775,
  0.8604729175567627,
  0.7525522708892822,
  0.6442756652832031,
  0.6509532928466797),
 'test_time': (0.15802454948425293,
  0.10372209548950195,
  0.11768436431884766,
  0.09175491333007812,
  0.08477425575256348)}

In [84]:
genres = df["variable"].unique()
res = {}
for genre in genres:
    res[genre] = [algo.predict(2753525, genre).est]
res = pd.DataFrame(data = res).T
res.sort_values(by=0, ascending=False)

,0
Role-Playing,1.708001
Action,1.510642
Strategy,0.892308
Simulation,0.278843
Misc,0.166420
Sports,0.149339
Fighting,0.133330
Puzzle,0.126299
Racing,0.115639
Platform,0.094984
